<a href="https://colab.research.google.com/github/RodolfoFigueroa/madi2022-1/blob/main/Unidad_2/5_Arboles_autobalanceables.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

En esta práctica, veremos una de las implementaciones más populares de árboles autobalanceables: el árbol rojo-negro.

# Definición

Decimos que un árbol binario es un árbol rojo-negro si cumple las siguientes propiedades:

1. Cada nodo tiene asignado un color: rojo o negro.
2. La raíz del árbol es negra.
3. Los hijos de un nodo rojo son negros.
4. Para cada nodo con al menos un hijo nulo (es decir, que no tiene al menos un hijo), el número de nodos negros desde cada nodo hasta cualquiera de sus descendientes nulos es el mismo para cualquiera de sus descendientes. A este número de nodos negros se le conoce como la *altura negra*.

# Representación
## Nodos

Para representar un nodo del árbol, usamos la siguiente estructura de datos:

In [ ]:
class RBNode:
    def __init__(self, val):
        self.red = False
        self.parent = None
        self.val = val
        self.left = None
        self.right = None

Con campos definidos de la siguiente manera:

* `red`: Si el nodo es rojo (`True`) o negro (`False`).
* `parent`: El padre del nodo.
* `val`: El valor del nodo.
* `left`: El hijo izquierdo.
* `right`: El hijo derecho.

## Árbol

Si bien en principio podemos utilizar tan solo los nodos para representar el árbol completo (de la forma que hicimos con BST), es más sencillo crear una segunda estructura para guardar todo el árbol. Esta tiene dos atributos:

* `nil`: El llamado "nodo sentinela" (explicado más adelante). Este nodo tiene `red=False`, y sus demás atributos `=None`.
* `root`: La raíz del árbol.

El algoritmo se vuelve mucho más sencillo si todos los nodos tienen dos hijos. Para garantizar esto, si a un nodo le falta un hijo, le asignamos como hijo el nodo sentinela de su árbol correspondiente. Por ejemplo, si tenemos el siguiente árbol:

```
      5
     /  \
    3    7
   /
  1
```

Con los nodos sentinelas (representados por `X`) se vuelve:

```
       5
     /   \
    3     7
   / \   / \
  1   X X   X
 / \
X   X
```

# Rotaciones
Las rotaciones son una operación sobre los árboles binarios que toman a un nodo como "pivote", y "rotan" a sus descendientes alrededor de él. La forma más fácil de ilustrarlas es con un diagrama:

## Rotación izquierda
En este caso, el pivote es `x`.

<img src="https://github.com/RodolfoFigueroa/madi2022-1/blob/main/Unidad_2/img/img6.jpg?raw=1" alt="Drawing" style="width: 600px;"/>

## Rotación derecha
En este caso, el pivote es `y`.

<img src="https://github.com/RodolfoFigueroa/madi2022-1/blob/main/Unidad_2/img/img7.jpg?raw=1" alt="Drawing" style="width: 600px;"/>

Podemos ver que la rotación izquierda es inversa de la derecha.

# Inserción

Los algoritmos que no alteran el árbol (búsqueda, caminata, etc.) son idénticos a los de un BST normal. Sin embargo, el algoritmo de inserción (y borrado) sufre muchos cambios, ya que al insertar el nuevo nodo, se deben de mantener las propiedades del árbol negro-rojo.

Para resolver esto, si queremos insertar un nodo nuevo `z`, le asignamos un color rojo (ya que de otra manera cambiaría la altura negra, lo cual sería difícil de resolver) y lo insertamos como si fuese un BST normal. Finalmente, llamamos una función `fix_insert` que modifica el árbol de modo que se sigan cumpliendo las propiedades.

Una vez insertado el nodo, existen dos casos posibles para el color de su padre:

1. Su padre es negro. En este caso, no tenemos que hacer nada, ya que todas las propiedades se siguen cumpliendo.
2. Su padre es rojo. Aquí, se viola la propiedad 3.

Ahora, asumiendo que ocurre el segundo caso, si el padre de `z` (llámese `p`) es un nodo izquierdo (el caso en el que es derecho es simétrico), pueden ocurrir 3 ordenamientos diferentes:

## Caso 1

<img src="https://github.com/RodolfoFigueroa/madi2022-1/blob/main/Unidad_2/img/img1.jpg?raw=1" alt="Drawing" style="width: 700px;"/>

Suponiendo que tenemos el primer caso, lo único que tenemos que hacer es "mover" el color rojo hacia arriba; de esta manera, la altura negra sigue siendo la misma.

<img src="https://github.com/RodolfoFigueroa/madi2022-1/blob/main/Unidad_2/img/img2.jpg?raw=1" alt="Drawing" style="width: 600px;"/>

Sin embargo, es posible que este desplazamiento haga que ahora el nodo padre viole la propiedad 3; por ejemplo, si el padre del nodo con valor 12 también es rojo. Para resolver esto, hacemos un nuevo ajuste, esta vez tomando al nodo padre como si fuese el nodo que insertamos.

## Casos 2 y 3

Nótese que podemos transformar el caso 2 en el caso 3 haciendo una rotación izquierda sobre el padre de `z`. Como tanto `z` como su padre son rojos, esta rotación no afectará la altura negra.

<img src="https://github.com/RodolfoFigueroa/madi2022-1/blob/main/Unidad_2/img/img4.jpg?raw=1" alt="Drawing" style="width: 400px;"/>

Una vez que tenemos el caso 3, coloreamos el padre de `z` negro, y su abuelo rojo, y luego hacemos una rotación hacia derecha sobre el abuelo de `z`.

<img src="https://github.com/RodolfoFigueroa/madi2022-1/blob/main/Unidad_2/img/img5.jpg?raw=1" alt="Drawing" style="width: 700px;"/>



In [ ]:
def print_tree(node, lines=None, level=0):
    if lines is None:
        lines = []
    if node.val is not None:
        print_tree(node.left, lines, level+1)
        lines.append(f"{' ' * 4 * level} {node.val} {'r' if node.red else 'b'}")
        print_tree(node.right, lines, level+1)
    else:
        lines.append(f"{' ' * 4 * level}  x")
    return '\n'.join(lines)

class RBTree:
    def __init__(self):
        self.nil = RBNode(None)
        self.root = self.nil

    def insert(self, val):
        new_node = RBNode(val)
        new_node.parent = None
        new_node.left = self.nil
        new_node.right = self.nil
        new_node.red = True

        parent = None
        current = self.root
        while current != self.nil:
            parent = current
            if new_node.val < current.val:
                current = current.left
            elif new_node.val > current.val:
                current = current.right
            else:
                return

        new_node.parent = parent
        if parent is None:
            self.root = new_node
        elif new_node.val < parent.val:
            parent.left = new_node
        else:
            parent.right = new_node

        self.fix_insert(new_node)

    def rotate_left(self, x):
        y = x.right
        x.right = y.left
        if y.left != self.nil:
            y.left.parent = x

        y.parent = x.parent
        if x.parent is None:
            self.root = y
        elif x == x.parent.left:
            x.parent.left = y
        else:
            x.parent.right = y
        y.left = x
        x.parent = y

    def rotate_right(self, x):
        y = x.left
        x.left = y.right
        if y.right != self.nil:
            y.right.parent = x

        y.parent = x.parent
        if x.parent == None:
            self.root = y
        elif x == x.parent.right:
            x.parent.right = y
        else:
            x.parent.left = y
        y.right = x
        x.parent = y
        
    def fix_insert(self, new_node):
        while new_node != self.root and new_node.parent.red:
            if new_node.parent == new_node.parent.parent.right: # si el padre es izquierdo
                u = new_node.parent.parent.left  # tío (hermano del padre)
                if u.red:
                    u.red = False
                    new_node.parent.red = False
                    new_node.parent.parent.red = True
                    new_node = new_node.parent.parent
                else:
                    if new_node == new_node.parent.left: # Caso 2 
                        self.rotate_right(new_node.parent)
                        new_node = new_node.parent
                    new_node.parent.red = False
                    new_node.parent.parent.red = True
                    self.rotate_left(new_node.parent.parent)
                    
            else: # si el padre es derecho
                u = new_node.parent.parent.right # tío (hermano del padre)

                if u.red:
                    u.red = False
                    new_node.parent.red = False
                    new_node.parent.parent.red = True
                    new_node = new_node.parent.parent
                else:
                    if new_node == new_node.parent.right:
                        new_node = new_node.parent
                        self.rotate_left(new_node)
                    new_node.parent.red = False
                    new_node.parent.parent.red = True
                    self.rotate_right(new_node.parent.parent)
        self.root.red = False

    def __repr__(self):
        return print_tree(self.root)

In [ ]:
tree = RBTree()
for x in range(1, 21):
    tree.insert(x)
print(tree)

                  x
             1 b
                  x
         2 b
                  x
             3 b
                  x
     4 r
                  x
             5 b
                  x
         6 b
                  x
             7 b
                  x
 8 b
                  x
             9 b
                  x
         10 b
                  x
             11 b
                  x
     12 r
                      x
                 13 b
                      x
             14 r
                      x
                 15 b
                      x
         16 b
                      x
                 17 b
                      x
             18 r
                      x
                 19 b
                          x
                     20 r
                          x


# Ejercicios

## Ejercicio 1

Investiga e implementa el algoritmo para borrar un nodo de un árbol rojo-negro. Utiliza en la medida de lo posible las funciones existentes (i.e. rotaciones)

Primero notamos que al borrar un nodo puede afectar las propiedades de un árbol rojo-negro, por lo que es importante encontrar un algoritmo para hacerlo bien.
### Primero vamos a hacer el algortimo para borrar un nodo 

1 escoger el nodo que sera borrado


2 gurdar en la memoria el color de ese nodo 

3 Si el nodo hijo izquierdo es nulo

- Le asignamos el nodo derecho a x
- transplantamos el nodo borrado con x

4  Si el nodo derccho es nulo hacemos el paso 3 pero ahora con el  izquerido

5  De otra forma 

- asignamos el minimo del subárbol derecho  a y

- salvamos el color original de y 

- asignamos el hijo derecho  de y a x

- si y es hijo del nodo que sera eliminado entonces ponemos como el padre de x a y

-De otra forma transplantmaos a y a los hijos derechos de y

- Le ponemos a y el color original

6 si el color original es negro llamamos DeleteFix(x)

## Algoritmo para mantener las propiedades del árbol rojo-negro

Este algorimto se implementa ya que cuando un nodo negro se borra  se rompen las reglas.
Pero esto se puede arreglar si asumimos que el nodo x  tiene un extra negro. Esto hace que el nodo x que no es rojo ni negro, que rompe los principios, pero el atributo de color de x no es cambiado en cambio el color extra negro es representado en x apuntando al nodo.

El nodo negro extra puede ser eliminado si

1 Alcanza al nodo raíz 

2 Si x apunta a un nodo rojo-negro , en este caso x es coloreado negro.

3Se hacen las rotaciones y los recoloramientos requeridos

Al seguir el algoritmo se conservan las propiedades del árbol rojo-negro.

1 Hacer lo siguiente hasta que x no sea la raíz del árbol y el color de x sea negro

2 Si x  es el hijo izquierdo 

- asignamos w al hermano de x

- si el hermano de x es rojo

   caso 1

   - Le asignamos el color del hijo derecho del padre de x como negro
   - Asignamos el color del padre de x como rojo
   - rotación izquierda del padre de x
   - Asignamos el hijo derecho del padre de x a w
- Si el color de ambos hijos de w es negro

   caso 2
  - Asignamos el color de w como rojo

  - Asignamos el padre de x a x
- Si el color de el hijo derecho de w es negro

  caso 3
  - Asignamos el color del hijo izquerdo de w como negro
  - Asignamos el color de w como rojo

  - rotación derecha de w
  - Asignamos el hijo derecho del padre de x a w
-Si ninguno de los casos pasados ocurre hacer lo siguiente
  caso 4 
  -Asignamos el color de w como el color del padre de x
  - Aasignamos el color del padre del padre de x como negro
  - Asignamos el color del hijo derecho de w como negro
  - Rotción izquierda del padre de x
  -Asignamos como x a la raíz del árbol
- Como arriba con el cambio izquierdo y vice versa
- Asignamos el color de x como negro


Y listo ya se borrara un nodo y se preservan las propiedades



## Referencias 
Tanto la explicación como el código lo saque de 

* https://www.programiz.com/dsa/deletion-from-a-red-black-tree





In [ ]:
import sys


# Node creation
class Node():
    def __init__(self, item):
        self.item = item
        self.parent = None
        self.left = None
        self.right = None
        self.color = 1


class RedBlackTree():
    def __init__(self):
        self.TNULL = Node(0)
        self.TNULL.color = 0
        self.TNULL.left = None
        self.TNULL.right = None
        self.root = self.TNULL

    # Preorder
    def pre_order_helper(self, node):
        if node != TNULL:
            sys.stdout.write(node.item + " ")
            self.pre_order_helper(node.left)
            self.pre_order_helper(node.right)

    # Inorder
    def in_order_helper(self, node):
        if node != TNULL:
            self.in_order_helper(node.left)
            sys.stdout.write(node.item + " ")
            self.in_order_helper(node.right)

    # Postorder
    def post_order_helper(self, node):
        if node != TNULL:
            self.post_order_helper(node.left)
            self.post_order_helper(node.right)
            sys.stdout.write(node.item + " ")

    # Search the tree
    def search_tree_helper(self, node, key):
        if node == TNULL or key == node.item:
            return node

        if key < node.item:
            return self.search_tree_helper(node.left, key)
        return self.search_tree_helper(node.right, key)

    # Balancing the tree after deletion
    def delete_fix(self, x):
        while x != self.root and x.color == 0:
            if x == x.parent.left:
                s = x.parent.right
                if s.color == 1:
                    s.color = 0
                    x.parent.color = 1
                    self.left_rotate(x.parent)
                    s = x.parent.right

                if s.left.color == 0 and s.right.color == 0:
                    s.color = 1
                    x = x.parent
                else:
                    if s.right.color == 0:
                        s.left.color = 0
                        s.color = 1
                        self.right_rotate(s)
                        s = x.parent.right

                    s.color = x.parent.color
                    x.parent.color = 0
                    s.right.color = 0
                    self.left_rotate(x.parent)
                    x = self.root
            else:
                s = x.parent.left
                if s.color == 1:
                    s.color = 0
                    x.parent.color = 1
                    self.right_rotate(x.parent)
                    s = x.parent.left

                if s.right.color == 0 and s.left.color == 0:
                    s.color = 1
                    x = x.parent
                else:
                    if s.left.color == 0:
                        s.right.color = 0
                        s.color = 1
                        self.left_rotate(s)
                        s = x.parent.left

                    s.color = x.parent.color
                    x.parent.color = 0
                    s.left.color = 0
                    self.right_rotate(x.parent)
                    x = self.root
        x.color = 0

    def __rb_transplant(self, u, v):
        if u.parent == None:
            self.root = v
        elif u == u.parent.left:
            u.parent.left = v
        else:
            u.parent.right = v
        v.parent = u.parent

    # Node deletion
    def delete_node_helper(self, node, key):
        z = self.TNULL
        while node != self.TNULL:
            if node.item == key:
                z = node

            if node.item <= key:
                node = node.right
            else:
                node = node.left

        if z == self.TNULL:
            print("Cannot find key in the tree")
            return

        y = z
        y_original_color = y.color
        if z.left == self.TNULL:
            x = z.right
            self.__rb_transplant(z, z.right)
        elif (z.right == self.TNULL):
            x = z.left
            self.__rb_transplant(z, z.left)
        else:
            y = self.minimum(z.right)
            y_original_color = y.color
            x = y.right
            if y.parent == z:
                x.parent = y
            else:
                self.__rb_transplant(y, y.right)
                y.right = z.right
                y.right.parent = y

            self.__rb_transplant(z, y)
            y.left = z.left
            y.left.parent = y
            y.color = z.color
        if y_original_color == 0:
            self.delete_fix(x)

    # Balance the tree after insertion
    def fix_insert(self, k):
        while k.parent.color == 1:
            if k.parent == k.parent.parent.right:
                u = k.parent.parent.left
                if u.color == 1:
                    u.color = 0
                    k.parent.color = 0
                    k.parent.parent.color = 1
                    k = k.parent.parent
                else:
                    if k == k.parent.left:
                        k = k.parent
                        self.right_rotate(k)
                    k.parent.color = 0
                    k.parent.parent.color = 1
                    self.left_rotate(k.parent.parent)
            else:
                u = k.parent.parent.right

                if u.color == 1:
                    u.color = 0
                    k.parent.color = 0
                    k.parent.parent.color = 1
                    k = k.parent.parent
                else:
                    if k == k.parent.right:
                        k = k.parent
                        self.left_rotate(k)
                    k.parent.color = 0
                    k.parent.parent.color = 1
                    self.right_rotate(k.parent.parent)
            if k == self.root:
                break
        self.root.color = 0

    # Printing the tree
    def __print_helper(self, node, indent, last):
        if node != self.TNULL:
            sys.stdout.write(indent)
            if last:
                sys.stdout.write("R----")
                indent += "     "
            else:
                sys.stdout.write("L----")
                indent += "|    "

            s_color = "RED" if node.color == 1 else "BLACK"
            print(str(node.item) + "(" + s_color + ")")
            self.__print_helper(node.left, indent, False)
            self.__print_helper(node.right, indent, True)

    def preorder(self):
        self.pre_order_helper(self.root)

    def inorder(self):
        self.in_order_helper(self.root)

    def postorder(self):
        self.post_order_helper(self.root)

    def searchTree(self, k):
        return self.search_tree_helper(self.root, k)

    def minimum(self, node):
        while node.left != self.TNULL:
            node = node.left
        return node

    def maximum(self, node):
        while node.right != self.TNULL:
            node = node.right
        return node

    def successor(self, x):
        if x.right != self.TNULL:
            return self.minimum(x.right)

        y = x.parent
        while y != self.TNULL and x == y.right:
            x = y
            y = y.parent
        return y

    def predecessor(self,  x):
        if (x.left != self.TNULL):
            return self.maximum(x.left)

        y = x.parent
        while y != self.TNULL and x == y.left:
            x = y
            y = y.parent

        return y

    def left_rotate(self, x):
        y = x.right
        x.right = y.left
        if y.left != self.TNULL:
            y.left.parent = x

        y.parent = x.parent
        if x.parent == None:
            self.root = y
        elif x == x.parent.left:
            x.parent.left = y
        else:
            x.parent.right = y
        y.left = x
        x.parent = y

    def right_rotate(self, x):
        y = x.left
        x.left = y.right
        if y.right != self.TNULL:
            y.right.parent = x

        y.parent = x.parent
        if x.parent == None:
            self.root = y
        elif x == x.parent.right:
            x.parent.right = y
        else:
            x.parent.left = y
        y.right = x
        x.parent = y

    def insert(self, key):
        node = Node(key)
        node.parent = None
        node.item = key
        node.left = self.TNULL
        node.right = self.TNULL
        node.color = 1

        y = None
        x = self.root

        while x != self.TNULL:
            y = x
            if node.item < x.item:
                x = x.left
            else:
                x = x.right

        node.parent = y
        if y == None:
            self.root = node
        elif node.item < y.item:
            y.left = node
        else:
            y.right = node

        if node.parent == None:
            node.color = 0
            return

        if node.parent.parent == None:
            return

        self.fix_insert(node)

    def get_root(self):
        return self.root

    def delete_node(self, item):
        self.delete_node_helper(self.root, item)

    def print_tree(self):
        self.__print_helper(self.root, "", True)


if __name__ == "__main__":
    bst = RedBlackTree()

    bst.insert(55)
    bst.insert(40)
    bst.insert(65)
    bst.insert(60)
    bst.insert(75)
    bst.insert(57)

    bst.print_tree()

    print("\nAfter deleting an element")
    bst.delete_node(40)
    bst.print_tree()

R----55(BLACK)
     L----40(BLACK)
     R----65(RED)
          L----60(BLACK)
          |    L----57(RED)
          R----75(BLACK)

After deleting an element
R----65(BLACK)
     L----57(RED)
     |    L----55(BLACK)
     |    R----60(BLACK)
     R----75(BLACK)


## Ejercicio 2

Escribe una función para convertir un árbol rojo-negro a una gráfica de `networkx`, de modo que cada nodo tenga como propiedad su color. Posteriormente, genera un árbol aleatorio, conviértelo a `networkx` y grafícalo de modo que los nodos mostrados tengan los colores correctos. Asegúrate que el árbol tenga el layout apropiado.

In [ ]:
# Aquí va tu código